Measurements

Measure from aircraft:
* indicated airspeed
* indicated altitude
* delta off centerline

Measure on ground:
* ambient temperature
* ambient pressure

From photo:
* aircraft true height *(using geometry as discussed)*

![037](pictures/tower_geometry_green2.png)


- calculate delta height from photos
- calculate ISA temperature for tower pressure
- from tower pressure, calculate tower pressure altitude
- from tower pressure altitude, delta height from photo and actual tower temp, calculate true pressure altitude at aircraft level (HPtrue)
- calculate pressure for HPtrue
- correct airspeed and alt for instument error
- convert indicated altitude to pressure
- calculate delta pressure altitude (HPtrue - HPindicated)
- calculate delta static pressure
- calculate delta dynamic pressure
- calculate delta airspeed
- compare to FAR limits

version control

version 1.0  - initial release
        1.01 - corrected text typos

Preamble:
* imports
* conversion factors

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
# constants and conversion factors
m2ft = 3.28084
ft2m = 1 / m2ft
kt2ms = 0.514444
ms2kt = 1 / kt2ms
C2K = 273.15

In [ ]:
# International Standard Atmosphere
T0 = 15 + C2K # (K)
p0 = 101325 # (Pa)
L = -6.5 / 1000 # (K/m)
a0 = 340.2 # (m/s)

rho0 = 1.225 # (kg/m3)
R = 287.05287 # (J/kg)/K
g_zero = 9.80665 # (m/s2)
gamma = 1.4 # adiabatic index for air

Tower Geometry Data

In [ ]:
# tower geometric data
cam_rwy = 384 # (m)
cam_twr = 520 # (m)
twr = 75 # (m)
cam_ref_h = 23 # (m)

Aircraft and Flight Test Data

In [ ]:
# data from airplane / instruments calibration
#  wingspan
b = 28.73 # (m)
#  speeds
Vmo = 320 # (kts)
Vsr0 = 105 # (kts)
#  instrument errors
delta_Vic = 1 # (kts)
delta_Hpic = 2 # (ft)

In [ ]:
# FT data

# from airplane
Vi = np.array([132, 185, 185, 230, 225]) # (KIAS)
Hpi = np.array([2015, 2040, 1970, 2085, 2075]) # (ft)
delta_centerline = np.array([0, 1.0, -0.25, 0, -0.5]) # fraction of wingspan

# from tower
p_amb_cam = np.array([95454.4, 95501.7, 95467.3, 95459.3, 95509.8]) # (Pa)
T_amb_cam = np.array([20, 20, 20, 20, 20]) # (C)
delta_acft_photo = np.array([87.9, 98.2, 47.9, 126.3, 117.6]) # (m)

# Calculate Aicraft $\Delta h$ - Aircraft Height Above Runway *(for each TP)*

$tan(\alpha)=\frac{(acft\_\Delta h - cam\_ref\_h)}{cam\_rwy \pm off\_centerline}=\frac{(twr-cam\_ref\_h)+\Delta acft\_photo}{cam\_twr}$

<p></p>
<p></p>
<p></p>

<font size="+2">    
$acft\_\Delta h = (cam\_rwy\pm off\_centerline) \frac{(twr-cam\_ref\_h)+\Delta acft\_photo}{cam\_twr} + cam\_ref\_h)$
</font>

In [ ]:
off_centerline = delta_centerline * b # (m)
acft_dh = ((cam_rwy + off_centerline) * ( (twr - cam_ref_h + delta_acft_photo) / cam_twr) + cam_ref_h) * m2ft # (ft)

In [ ]:
np.set_printoptions(precision=0)
print(f'Aircraft delta_h: {acft_dh} ft')

# Find the ISA temperature at the camera position, from its ambient pressure 

from ambient pressure, measured at the camera position:

<font size="+2"> 
    
$T_{std}=T_{0}(\frac{p_{amb_{cam}}}{p_{0}})^{-\frac{R*L}{g_{0}}}  $
</font>

In [ ]:
T_std_cam = T0 * ((p_amb_cam) / p0)**(- (R * L) / g_zero) # (K)
print(f'Standard temperature at camera level: {T_std_cam} K') 

# Calculate pressure altitude at the camera position 

also from ambient pressure, measured at the camera position:

<font size="+2"> 
    
$H_{Pcam}=\frac{T_{0}}{L} * ((\frac{p_{amb_{cam}}}{p_{0}})^{\frac{-R*L}{g_{0}}}-1)$  *note units: $H_{Pic}$ in meters*
</font>

In [ ]:
Hp_cam = T0 / L * ((p_amb_cam / p0)**(- (R * L) / g_zero) -1) * m2ft # (ft)
print(f'Pressure altitude at camera: {Hp_cam} ft')

## Pressure Altitude at Aircraft Passing Level

From the pressure altitude at the camera position, we need to add the delta altitude to reach the aircraft passing level. We have the aircraft height $\Delta h$ (tapeline), and to transform that into delta altitude, we need to consider the difference in standard and test day temperatures:

<font size="+2"> 
$H_{Pc}=Hpcam+\Delta h \frac{T_{std}}{T_{test}}$

In [ ]:
T_amb_cam_K = T_amb_cam + C2K # (K)
Hpc = Hp_cam + acft_dh * (T_std_cam / T_amb_cam_K) # (ft)
print(f'Pressure altitude at aircraft passing: {Hpc} ft')

## Calculate static pressure for $H_{Pc}$

<font size="+2"> 
$p_{static_{true}} = p_0 * (1+\frac{L}{T_0}*(H_{Pc})^{\frac{-g_0}{RL}})$

In [ ]:
p_static_true = p0 * (1 + L / T0 * (Hpc*ft2m))**(-g_zero / ( R * L)) # (Pa)
print(f'Static pressure at aircraft passing level: {p_static_true} Pa')

# Aircraft Side
## Correct for Instrument Errors

$V_{ic}=V{i}+\Delta V_{ic}$

$H_{Pic}=H_{Pi}+\Delta H_{Pic}$

In [ ]:
Vic = Vi + delta_Vic # (kts)
Hpic = Hpi + delta_Hpic # (ft)

# Convert $Hp_{ic}$ to Static Pressure, per International Standard Atmosphere
at the aircraft static port:

<font size="+2"> 
$p_{static_{acft}} = p_0 * (1+\frac{L}{T_0}*(H_{Pic})^{\frac{-g_0}{RL}})$  *note units: $H_{Pic}$ in meters*

In [ ]:
p_static_acft = p0 * (1 + L /T0 * (Hpic*ft2m))**(-g_zero / (R * L)) # (Pa)
print(f'Static pressure from aircraft instruments: {p_static_acft} Pa')

# Error Calculation

## Altitude Error

<font size="+2"> 
$\Delta H_{Pc} = H_{Pc} - H_{Pic}$


<font size="+0">   
 -*Positive $\Delta H_{Pc}$ translates to aircraft indicated altitude lower than true value.*-

In [ ]:
delta_Hpc = Hpc - Hpic # (ft)
np.set_printoptions(precision=1)
print(f'Altitude error: {delta_Hpc} ft')

## Calculate the $\Delta$ in static pressures (true - aircraft)

i.e., the static pressure error correction

<font size="+2">
$\Delta p_{s} = p_{static_{true}} - p_{static_{acft}} $

In [ ]:
delta_ps = p_static_true - p_static_acft # (Pa)
print(f'Static pressure error: {delta_ps} Pa')

## Speed Error

Since we assumed all error in on the static port, we can evaluate what will be the indicated airspeed due to the static port error. Remember that the pitot measures total pressure...

<font size="+2">
$\Delta p_{d} = -\Delta p_{s}$

In [ ]:
delta_pd = - delta_ps

## Translate $\Delta p_{d}$ into speed error $\Delta V_{pc}$

<font size=5>$\Delta V_{pc} = \frac{a_{0}^{2} \frac{p_{static_{true}}}{p_0}}{\gamma*V_{ic}*(1+0.2*(\frac{V_{ic}}{a_{0}})^2)^{2.5}}*\frac{\Delta p_{d}}{p_s}$

In [ ]:
delta_Vpc_SI= a0**2 * (p_static_true / p0) / (gamma * (Vic * kt2ms) * (1 + 0.2 * ((Vic * kt2ms) / a0)**2)**2.5) * delta_pd / p_static_acft # (m/s)
delta_Vpc = delta_Vpc_SI * ms2kt # (kts)
np.set_printoptions(precision=4)
print(f'Airspeed error: {delta_Vpc} kt')

# FAR 25.1323 Compliance Check

## Speed

25.1323 (c) states:
The airspeed error of the installation, excluding the airspeed indicator instrument calibration error, may not exceed three percent or five knots, whichever is greater, throughout the speed range, from - 

(1) VMO  to 1.23 VSR1, with flaps retracted; and 

(2) 1.23 VSR0 to VFE with flaps in the landing position. 

In [ ]:
k_point = 5 / 0.03 # (kts)
x_limit1 = np.linspace(Vsr0, k_point, 10)
top_limity1 = np.ones(x_limit1.shape[0]) * 5
bot_limity1 = top_limity1 * -1
x_limit2 = np.linspace(k_point, Vmo, 10)
top_limity2 = x_limit2 * 0.03
bot_limity2 = top_limity2 * -1

In [ ]:
plt.rcParams['figure.figsize'] = [12, 7]
# plot limits
plt.plot(x_limit1, top_limity1, 'r', label='FAR 25 limit')
plt.plot(x_limit1, bot_limity1, 'r')
plt.plot(x_limit2, top_limity2, 'r')
plt.plot(x_limit2, bot_limity2, 'r')
# plot data
plt.plot(Vic, delta_Vpc, '+b', label='Flight Test', markersize=10)

plt.xlabel('Vic (kts)')
plt.ylabel('delta Vpc (kts)')
plt.title('Speed Error Plot')
plt.ylim(-10, 10)
plt.grid(True)
plt.legend()
plt.show()

## Altitude - 25.1325
(e) Each system must be designed and installed so that the error in indicated pressure altitude, at sea level, with a standard atmosphere, excluding instrument calibration error, does not result in an error of more than ±30 feet per 100 knots speed for the appropriate configuration in the speed range between 1.23 VSR0 with flaps extended and 1.7 VSR1 with flaps retracted. However, the error need not be less than ±30 feet. 

In [ ]:
k_point = 100 # (kts)
x_limit1 = np.linspace(0, k_point, 10)
top_limity1 = np.ones(x_limit1.shape[0]) * 30
bot_limity1 = top_limity1 * -1
x_limit2 = np.linspace(k_point, Vmo, 10)
top_limity2 = x_limit2 * 0.3
bot_limity2 = top_limity2 * -1

In [ ]:

# plot limits
plt.plot(x_limit1, top_limity1, 'r', label='FAR 25 limit')
plt.plot(x_limit1, bot_limity1, 'r')
plt.plot(x_limit2, top_limity2, 'r')
plt.plot(x_limit2, bot_limity2, 'r')
# plot data
plt.plot(Vic, delta_Hpc, '+b', label='Flight Test', markersize=10)

plt.xlabel('Vic (kts)')
plt.ylabel('delta Hpc (ft)')
plt.title('Altitude Error Plot')
#plt.ylim(-10, 10)
plt.grid(True)
plt.legend()
plt.show()